# RSA密码破译

`Import the necesssary module`

In [3]:
import gmpy2
from Crypto.Util.number import *
import owiener

`Load data`

In [4]:
def load_data(datanum):
    with open(file="./test/data%d" % (datanum), mode='r') as file:
        n, e, c = file.read(256), file.read(256), file.read(256)
        n, e, c = int(n, 16), int(e, 16), int(c, 16)
        return n, e, c


`Attack RSA`

In [6]:
def attack_broadcast(e,lst_c,lst_n):
    M=1
    for n in lst_n:
        M=M*n
    lst_M=[]
    for n in lst_n:
        lst_M+=[M//n]
    m_e=M
    for i in range(len(lst_c)):
        m_e+=lst_c[i]*lst_M[i]*inverse(lst_M[i],lst_n[i])
        print(i)
    m_e=m_e%M
    m=gmpy2.iroot(m_e,e)
    i=0
    while not m[1]:
        m_e+=M
        i+=1
        m=gmpy2.iroot(m_e,e)
    return m

In [7]:
lst_n,lst_c=[],[]
for i in range(21):
    n,e,c=load_data(i)
    if e==5:
        lst_n+=[n]
        lst_c+=[c]
attack_broadcast(5,lst_c,lst_n)

0
1
2
3
4


(mpz(7985094500508197619216095179109374591466840832845633905102428354228364957393952749844260287135965730215484818378063570600824813771406220340542500306493542),
 True)

In [10]:
n_lst=[]
for i in range(21):
    n,e,c=load_data(i)
    print(e,i)

3 0
65537 1
65537 2
65537 3
5 4
5 5
3 6
5 7
65537 8
65537 9
65537 10
46786465362686334917265996843779843233606992585424976481745055335468678697948774988450305612127967926533923268260412557000125153569622340353246096040604284883505587337829322949633637609180797447754513992039018904786537115087888005528547900640339270052628915440787357271345416818313808448127098885767015748889 11
65537 12
5 13
152206992575706893484835984472544529509325440944131662631741403414037956695665533186650071476146389737020554215956181827422540843366433981607643940546405002217220286072880967331118344806315756304650248634546597784597963886656422706197757265316981889118026978865295597135470735576032282694348773714479076093197 14
65537 15
65537 16
3 17
5 18
65537 19
65537 20


In [9]:
def fmt(n):
    for i in range(100000):
        m=gmpy2.iroot(n+i*i,2)
        if m[1]:
            return 1
    return 0
    